In [1]:
# Tokens for connection to different services
from clan_tools.secrets.Vault import Vault
Vault().get_secrets()

# Connector to CHYT
from clan_tools.data_adapters.ClickHouseYTAdapter import ClickHouseYTAdapter
ch_adapter = ClickHouseYTAdapter()

# Connector to YT 
from clan_tools.data_adapters.YTAdapter import YTAdapter
yt_adapter = YTAdapter()

# Connector to YQL
from clan_tools.data_adapters.YQLAdapter import YQLAdapter
yql_adapter = YQLAdapter()

In [2]:
import pandas as pd
import requests as req

In [3]:
# AWS IP from https://docs.aws.amazon.com/general/latest/gr/aws-ip-ranges.html#aws-ip-download
url = 'https://ip-ranges.amazonaws.com/ip-ranges.json'
aws_ip_json = req.get(url).json()

In [8]:
aws_ip_df = pd.DataFrame(aws_ip_json['prefixes'])
aws_ip_df.head()

,ip_prefix,region,service,network_border_group
0,3.5.140.0/22,ap-northeast-2,AMAZON,ap-northeast-2
1,13.34.37.64/27,ap-southeast-4,AMAZON,ap-southeast-4
2,13.34.65.64/27,il-central-1,AMAZON,il-central-1
3,13.34.66.0/27,us-east-1,AMAZON,us-east-1
4,15.230.221.0/24,us-east-1,AMAZON,us-east-1


In [9]:
# Data downloading from df to YT
df_schema = [
    {"name": "ip_prefix", "type": "string"},
    {"name": "region", "type": "string"},
    {"name": "service", "type": "string"},
    {"name": "network_border_group", "type": "string"}
]

yt_adapter.save_result('//home/cloud_analytics/data_swamp/projects/ipv4/big3_ip_groups/aws',
                        df=aws_ip_df,
                        schema=df_schema,
                        append=False)


2022-07-07 14:31:58,644	WARNING	@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
2022-07-07 14:31:58,645	WARNING	@         WARNING: UNPROTECTED TOKEN KEY FILE!            @
2022-07-07 14:31:58,645	WARNING	@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
2022-07-07 14:31:58,646	WARNING	Permissions 0664 for '/home/daniilkhar/.yt/token' are too open.
2022-07-07 14:31:58,646	WARNING	It is required that your token file is NOT accessible by others.


In [13]:
# GCP IP from https://www.gstatic.com/ipranges/cloud.json
url = 'https://www.gstatic.com/ipranges/cloud.json'
gcp_ip_json = req.get(url).json()
gcp_ip_df = pd.DataFrame(gcp_ip_json['prefixes'])
gcp_ip_df.head()

,ipv4Prefix,service,scope,ipv6Prefix
0,34.80.0.0/15,Google Cloud,asia-east1,NaN
1,34.137.0.0/16,Google Cloud,asia-east1,NaN
2,35.185.128.0/19,Google Cloud,asia-east1,NaN
3,35.185.160.0/20,Google Cloud,asia-east1,NaN
4,35.187.144.0/20,Google Cloud,asia-east1,NaN


In [15]:
# Data downloading from df to YT
df_schema = [
    {"name": "ipv4Prefix", "type": "string"},
    {"name": "service", "type": "string"},
    {"name": "scope", "type": "string"},
    {"name": "ipv6Prefix", "type": "string"}
]

yt_adapter.save_result('//home/cloud_analytics/data_swamp/projects/ipv4/big3_ip_groups/gcp',
                        df=gcp_ip_df,
                        schema=df_schema,
                        append=False)

In [34]:
# Azure IP from https://download.microsoft.com/download/7/1/D/71D86715-5596-4529-9B13-DA13A5DE5B63/ServiceTags_Public_20220704.json
url = 'https://download.microsoft.com/download/7/1/D/71D86715-5596-4529-9B13-DA13A5DE5B63/ServiceTags_Public_20220704.json'
azure_ip_json = req.get(url).json()
azure_ip_df = pd.DataFrame.from_dict(azure_ip_json['values'])
azure_ip_df['ip_prefix'] = [d.get('addressPrefixes') for d in azure_ip_df.properties]
azure_ip_df.drop('properties', axis=1, inplace=True)
azure_ip_df = azure_ip_df.explode('ip_prefix')
azure_ip_df.head()

,name,id,ip_prefix
0,ActionGroup,ActionGroup,13.65.25.19/32
0,ActionGroup,ActionGroup,13.66.60.119/32
0,ActionGroup,ActionGroup,13.66.143.220/30
0,ActionGroup,ActionGroup,13.66.202.14/32
0,ActionGroup,ActionGroup,13.66.248.225/32


In [35]:
# Data downloading from df to YT
df_schema = [
    {"name": "name", "type": "string"},
    {"name": "id", "type": "string"},
    {"name": "ip_prefix", "type": "string"}
]

yt_adapter.save_result('//home/cloud_analytics/data_swamp/projects/ipv4/big3_ip_groups/azure',
                        df=azure_ip_df,
                        schema=df_schema,
                        append=False)